# 1. Cartpole System Evaluation

In [1]:
import gym
import tensorflow as tf
import numpy as np
import pickle

import control
from scipy.linalg import lu
import math
import cmath

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
model = tf.keras.models.load_model(
    './cartpole_system_model', custom_objects=None, compile=True, options=None
)

2022-04-27 00:54:45.803611: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
np_weights = model.get_weights()
print(model.A.get_weights())
print(model.B.get_weights())

[array([[-1.5739264e-08, -3.6348650e-04, -1.6419442e-08,  1.4684309e-03],
       [ 1.9999964e-02,  1.8883319e-04, -1.0376565e-08, -1.3297781e-03],
       [-6.3428889e-09, -1.3173890e-02,  1.2812671e-08,  3.1240752e-01],
       [ 2.3316607e-08,  1.7133448e-04,  2.0000000e-02, -5.5705023e-04]],
      dtype=float32)]
[array([[-1.7497801e-08,  1.9511257e-01, -6.7852040e-09, -2.9143080e-01]],
      dtype=float32)]


In [4]:
A = np_weights[0]
B = np_weights[1].T
print("A Matrix")
print(A)
print("B Matrix")
print(B)

A Matrix
[[-1.5739264e-08 -3.6348650e-04 -1.6419442e-08  1.4684309e-03]
 [ 1.9999964e-02  1.8883319e-04 -1.0376565e-08 -1.3297781e-03]
 [-6.3428889e-09 -1.3173890e-02  1.2812671e-08  3.1240752e-01]
 [ 2.3316607e-08  1.7133448e-04  2.0000000e-02 -5.5705023e-04]]
B Matrix
[[-1.7497801e-08]
 [ 1.9511257e-01]
 [-6.7852040e-09]
 [-2.9143080e-01]]


## Controllability

In [5]:
# controllability matrix
C_matrix = control.ctrb(A, B)

# obtain rank via LU decomposition
_, L_matrix, U_matrix = lu(C_matrix, permute_l = False)

print(L_matrix)
print(U_matrix)

[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 2.32823845e-08  1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-6.69498798e-01 -5.93330927e-03  1.00000000e+00  0.00000000e+00]
 [ 6.00410131e-08  5.32888703e-03  1.27628800e-04  1.00000000e+00]]
[[-2.91430801e-01  1.95770969e-04 -1.87234764e-03  2.15261888e-06]
 [ 0.00000000e+00 -9.36155664e-02  5.55684132e-05 -5.84801688e-04]
 [ 0.00000000e+00  0.00000000e+00 -1.26336133e-03  4.61950197e-07]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  3.70560843e-07]]


The controllability matrix is fully-ranked column-wise.

## Controllability Gramian

In [6]:
# gramian can be derived from the controllability matrix
W_c = np.matmul(C_matrix, C_matrix.T)
print(W_c)

lambda_wc, xi_wc = np.linalg.eig(W_c)
for i in range(len(lambda_wc)):
    print("Eigenvalue: ", i)
    print(lambda_wc[i])
    print("Eigenvector")
    print(xi_wc[i])

[[ 2.48875608e-07 -2.15132164e-07  4.67033081e-05 -9.28224680e-08]
 [-2.15132164e-07  3.80690956e-02 -3.97320761e-05 -5.68617108e-02]
 [ 4.67033081e-05 -3.97320761e-05  8.76421936e-03 -1.84305349e-05]
 [-9.28224680e-08 -5.68617108e-02 -1.84305349e-05  8.49354559e-02]]
Eigenvalue:  0
0.12300323620534576
Eigenvector
[-3.68482309e-07 -5.32872039e-03 -9.99985786e-01  1.80061230e-04]
Eigenvalue:  1
0.008764681482166175
Eigenvector
[5.56317654e-01 4.06988785e-03 1.27733160e-04 8.30959739e-01]
Eigenvalue:  2
1.3730605092133755e-13
Eigenvector
[-5.94232729e-05 -9.99973611e-01  5.32954444e-03  4.93665061e-03]
Eigenvalue:  3
1.1020438465338832e-06
Eigenvector
[-8.30969713e-01  2.79622000e-03  8.55771145e-05  5.56310624e-01]


Looking at the eigenvalues of the controllable gramian, we can see that, as expected, the pole's angle and angular velocities are less controllable compared to the cart's position and velocity.

Visualization of the gramian is abridged, since it requires a projection of a 4-dimensional ellipsoid onto a 2-dimensional plane, which is out of the scope of this project.

In [11]:
# def gramian_2d_ellipoid_projection(eig_x, eig_y, eig_vec): 
#     # parameterized ellipsOID function
#     # IMPORTANT: Cut the ellipsoid along a plane
#     return cut_gramian

In [12]:
# cart_gramian_ellipse = gramian_2d_ellipse(lambda_wc[0], lambda_wc[1], xi_wc[0])
# measure eigenvector sizes projected onto xy-plane